In [ ]:
import pandas as pd
import numpy as np
import joblib
import sklearn
from sklearn import metrics, utils, ensemble
from joblib import Parallel, delayed
import time 
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm 

In [ ]:
###########
# TODO: CHANGE INSTITUTION NAME
institution = 'INSTITUTION'

In [ ]:
# Input:  {INSTITUTION}-MASTER.csv
# Output: {INSTITUTION}-RESULTS.csv

In [ ]:
# Loads in data
master = pd.read_csv('{}-MASTER.csv'.format(institution))

req_cols = set(['hosp_id', 'y', 'y_score_fourvar', 'y_score_mcures',
                'y_scores_four_lst', 'y_scores_mcures_lst', 'race', 'age', 'sex', 'ethnicity',
                'outcome', 'outcome_time', 'admission_date',
                'final_time_min'])

master = master[req_cols]

In [ ]:
# Parse outcome scores for primary use case
from ast import literal_eval

master["y_scores_mcures_lst_"] = master["y_scores_mcures_lst"].apply(literal_eval)
master["y_scores_mcures_lst_eval1"] = master["y_scores_mcures_lst_"].apply(lambda L: L[1:]) # Exclude first window

master["y_scores_four_lst_"] = master["y_scores_four_lst"].apply(literal_eval)
master["y_scores_four_lst_eval1"] = master["y_scores_four_lst_"].apply(lambda L: L[1:]) # Exclude first window

In [ ]:
# Compute scores for secondary use case
in_second_case = (master['final_time_min'] > 2880).values
master['in_second_use_case'] = in_second_case

secondary_four = [np.mean(x[0:12]) if master['in_second_use_case'][i] else np.nan for i,x in enumerate(master['y_scores_four_lst_'].values)]
mcures_four = [np.mean(x[0:12]) if master['in_second_use_case'][i] else np.nan for i,x in enumerate(master['y_scores_mcures_lst_'].values)]

master['secondary_four'] = secondary_four
master['secondary_mcures'] = mcures_four

In [ ]:
#secondary use case y value 

master['y_secondary'] = (1 - np.isnan(master['outcome_time'])).values


In [ ]:
# Error checking helper functions
def check_allowed_categories(observed, allowed):
    observed = set(observed)
    allowed = set(allowed)
    intersection = observed.intersection(allowed)
    
    if observed != intersection:
        non_allowed = observed-allowed
        print("observed non-allowed categories: {}".format(', '.join([str(i) for i in non_allowed])))
        return(False)
    else:
        return(True)

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def unpack_lists(col_as_list):
    unpacked_list = []
    for i in col_as_list:
        #print(i)
        if type(i)==list:
            i = ','.join([str(j) for j in i])
        i = replace_all(i, {'[': '', ']':''})
        i = i.strip().split(',')
        i = [float(j.strip()) for j in i]
        unpacked_list.append(i)
    
    return(unpacked_list)

def check_list_lengths(lol1, lol2):
    #lol = list of lists
    for i,j in zip(lol1, lol2):
        if len(i)!=len(j):
            return False
    return True

def check_reference(lol, ref, list_op=max, eps=None):
    for idx, (i,j) in enumerate(zip(lol, ref)):
        _val = list_op(i)
        if eps is None:
            if _val!=j:
                print(idx, '\n', i, '\n' , j, _val, j in i, '\n')
                return False
        elif abs(_val-j)>eps:
            print(idx, '\n', i, '\n' , j, _val, j in i, '\n')
            return False
            
    return True


print("All Columns")
req_cols = set(['hosp_id', 'y', 'y_score_fourvar', 'y_score_mcures',
                'y_scores_four_lst', 'y_scores_mcures_lst', 'race', 'age', 'sex',
                'outcome', 'outcome_time', 'admission_date',
                'final_time_min'])

r = (req_cols == set(master.columns).intersection(req_cols))
print("\tHave all required columns? {}".format(r))


print("Age")
r = (master["age"].dtype == int)
print("\tIs integer? {}".format(r))
r = (master["age"].min()>=18)
print("\tMin>=18? {}".format(r))
r = (master["age"].max()<=90)
print("\tMax<=90? {}".format(r))

print("Encounters w/ Outcome")
_ = master[master["y"]==1]
r = ( (_["final_time_min"] - _["outcome_time"]).sum() == 0)
print("\tfinal_time equals outcome_time? {}".format(r))
allowed_outcomes = ['HHFNC', 'MV', 'mortality', 'IV']
r = check_allowed_categories(_["outcome"].unique(), allowed_outcomes)
print("\tOnly use allowed outcomes? {}".format(r))

print("Race")
allowed_races = ["African American", "American Indian or Alaska Native", "Asian", "Caucasian", "Native Hawaiian and Other Pacific Islander", "Other", "Patient Refused", "Unknown", "More than 1", np.nan]
r = check_allowed_categories(master["race"].unique(), allowed_races)
print("\tOnly use allowed race categories? {}".format(r))

print("Sex")
allowed_sexes = ["F", "M"]
r = check_allowed_categories(master["sex"].unique(), allowed_sexes)
print("\tOnly use allowed sex categories? {}".format(r))

print("Ethnicity")
allowed_ethnicities = ["Hispanic or Latino", "Non-Hispanic or Latino", "Patient Refused", "Unknown"]
# r= check_allowed_categories(master["ethnicity"].unique(), allowed_ethnicities)
#print("\tOnly use allowed ethnicity categories? {}".format(r))
print("\tUncomment above to do check.")

list_cns = ["y_scores_four_lst", "y_scores_mcures_lst", "y_scores_mcures_lst_", "y_scores_mcures_lst_eval1", "y_scores_four_lst_", "y_scores_four_lst_eval1"]
unpacked_list_dict = {cn: unpack_lists(list(master[cn])) for cn in list_cns}
nwin = [min(i, 30) for i in list(np.floor(master["final_time_min"]/(4*60)))]

print("Lists")
r = check_list_lengths(unpacked_list_dict["y_scores_four_lst"], unpacked_list_dict["y_scores_mcures_lst"])
print("\tScore lists the same length? {}".format(r))
r = check_list_lengths(unpacked_list_dict["y_scores_four_lst_eval1"], unpacked_list_dict["y_scores_mcures_lst_eval1"])
print("\tScore (eval1) lists the same length? {}".format(r))

r = check_reference(unpacked_list_dict["y_scores_four_lst_eval1"], list(master["y_score_fourvar"]), eps=1E-10)
print("\tIs fourvar (eval1) max is max of list? {}".format(r))
r = check_reference(unpacked_list_dict["y_scores_mcures_lst_eval1"], list(master["y_score_mcures"]), eps=1E-10)
print("\tIs MCURES (eval1) max is max of list? {}".format(r))

r = check_reference(unpacked_list_dict["y_scores_four_lst"], nwin, list_op=len)
print("\tIs len of fourvar equal to expected number of windows? {}".format(r))
r = check_reference(unpacked_list_dict["y_scores_four_lst_eval1"], [i-1 for i in nwin], list_op=len)
print("\tIs len of fourvar (eval1) equal to expected number of windows-1? {}".format(r))
r = check_reference(unpacked_list_dict["y_scores_mcures_lst"], nwin, list_op=len)
print("\tIs len of MCURES equal to expected number of windows? {}".format(r))
r = check_reference(unpacked_list_dict["y_scores_mcures_lst_eval1"], [i-1 for i in nwin], list_op=len)
print("\tIs len of MCURES (eval1) equal to expected number of windows-1? {}".format(r))

In [ ]:
#Helper Functions

def get_nums(df, string = 'y_score_mcures'): 
    N, perc, val_roc, all_rocs = get_roc(df['y'], df[string])
    all_prs, val_pr = get_pr(df['y'], df[string])
    return (N, perc), (all_rocs, val_roc), (all_prs, val_pr)

def get_roc(y_true, y_score):
    roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func_roc)(i, y_true, y_score) for i in range(1000)))
    val = metrics.roc_auc_score(y_true, y_score)
    return len(y_true), np.mean(y_true), val, auc_scores

def bootstrap_func_roc(i, y_true, y_score):
    while True:
        try:
            yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
            return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)
        except: 
            i += 1000

def bootstrap_func_calib(i, combine_all_scores, all_ys):
    yte_true_b, yte_pred_b = utils.resample(combine_all_scores, all_ys, replace=True, random_state=i)
    flat_ys = [item for sublist in yte_pred_b for item in sublist]
    flat_scores = [item for sublist in yte_true_b for item in sublist]
    fraction_of_positives, mean_predicted_value = calibration_curve(flat_ys, flat_scores, strategy = 'quantile')
    return fraction_of_positives, mean_predicted_value 

            
def get_calibs(combine_all_scores, all_ys):
    fractions, means = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func_calib)(i, combine_all_scores, all_ys) for i in range(1000)))
    return fractions, means
    
def get_roc_CI(y_true, y_score):
    roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func_roc)(i, y_true, y_score) for i in range(1000)))
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    for fpr, tpr, _ in roc_curves:
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        aucs.append(metrics.auc(fpr, tpr))
    mean_tpr = np.mean(tprs, axis=0)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + 1.96 * std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - 1.96 * std_tpr, 0)
    return roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper

def bootstrap_func_pr(i, y_true, y_score):
    while True:
        try:
            yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
            return (
                metrics.precision_recall_curve(yte_true_b, yte_pred_b), 
                metrics.auc(*metrics.precision_recall_curve(yte_true_b, yte_pred_b)[1::-1])
            )
        except: 
            i += 1000

def get_pr(y_true, y_score):
    curves, scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func_pr)(i, y_true, y_score) for i in range(1000)))
    val = metrics.auc(*metrics.precision_recall_curve(y_true, y_score)[1::-1])
    return scores, val

def get_pr_CI(y_true, y_score):
    curves, scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func_pr)(i, y_true, y_score) for i in range(1000)))
    precs = []
    mean_rec = np.linspace(0, 1, 101)
    for prec, rec, _ in curves:
        rec_sorted, prec_sorted = rec[np.argsort(rec)], prec[np.argsort(rec)]
        precs.append(np.interp(mean_rec, rec_sorted, prec_sorted))
    mean_prec = np.mean(precs, axis=0)
    std_prec = np.std(precs, axis=0)
    prec_upper = np.minimum(mean_prec + 1.96 * std_prec, 1)
    prec_lower = np.maximum(mean_prec - 1.96 * std_prec, 0)
    return curves, scores, mean_rec, prec_lower, prec_upper

from sklearn.utils import column_or_1d
from sklearn.preprocessing import label_binarize
from sklearn.utils.validation import check_consistent_length

def calibration_curve_ece(y_true, y_prob, *, normalize=False, n_bins=5,
                      strategy='uniform'):
    y_true = column_or_1d(y_true)
    y_prob = column_or_1d(y_prob)
    check_consistent_length(y_true, y_prob)

    if normalize:  # Normalize predicted values into interval [0, 1]
        y_prob = (y_prob - y_prob.min()) / (y_prob.max() - y_prob.min())
    elif y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError("y_prob has values outside [0, 1] and normalize is "
                         "set to False.")

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError("Only binary classification is supported. "
                         "Provided labels %s." % labels)
    y_true = label_binarize(y_true, classes=labels)[:, 0]

    if strategy == 'quantile':  # Determine bin edges by distribution of data
        quantiles = np.linspace(0, 1, n_bins + 1)
        bins = np.percentile(y_prob, quantiles * 100)
        bins[-1] = bins[-1] + 1e-8
    elif strategy == 'uniform':
        bins = np.linspace(0., 1. + 1e-8, n_bins + 1)
    else:
        raise ValueError("Invalid entry to 'strategy' input. Strategy "
                         "must be either 'quantile' or 'uniform'.")

    binids = np.digitize(y_prob, bins) - 1
    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    nonzero = bin_total != 0
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]

    return np.sum((np.bincount(binids) / sum(np.bincount(binids))) * np.abs(prob_true - prob_pred))


import copy

def fsgte(score_fx, df, thresholds=None):
    given_thresholds = thresholds
    _thresholds = []
    score_dict = {}
    label_dict = {}
    
    # for each hosp_id, extract a list of monotonically increasing scores
    for _, row in df.iterrows():
        _pid = row["hosp_id"]
        label_dict[_pid] = float(row["y"])
        _scores = row["y_scores_mcures_lst_eval1"]
        _thresholds += _scores
        i = _scores[0]
        monotonic_scores = [i]
        if len(_scores) > 1:
            for j in _scores[1:]:
                if j > i:
                    monotonic_scores.append(j)
                    i = j
        score_dict[_pid] = monotonic_scores
    
    # list of decision thresholds
    thresholds = _thresholds if thresholds is None else thresholds    
    thresholds = set(thresholds)
    thresholds.update({0,1})
    thresholds = sorted(thresholds)
    thresholded_score_dict = copy.deepcopy(score_dict)
    
    # for each hosp_id, extract the y label
    label_list = []
    for pid, _ in thresholded_score_dict.items():
        label_list.append(label_dict[pid])
    
    # calculate per-threshold calibration ece score
    scores_arr = []
    for threshold in thresholds:
        score_list = []
        for pid, scores in thresholded_score_dict.items():
            if len(scores) > 1:
                _scores = [i for i in scores if i>=threshold]
                thresholded_score_dict[pid] = _scores
            first_score = thresholded_score_dict[pid][0]
            score_list.append(first_score)
        scores_arr.append(score_fx(label_list, score_list))
        
    if given_thresholds is None:
        return thresholds, scores_arr
    else: 
        return scores_arr
    
def bootstrap_ece(i, function, df, thresholds):
    while True:
        try:
            df_resample = df.sample(frac=1, replace=True, random_state=i)
            return fsgte(function, df_resample, thresholds)
        except: 
            i += 1000
    return

def get_ece_CI(df):
    thresholds, scores = fsgte(calibration_curve_ece, master)
    ece_curves = np.array(Parallel(n_jobs=50)(delayed(bootstrap_ece)(i, calibration_curve_ece, df, thresholds) for i in range(100)))
    auece_scores = [metrics.auc(thresholds, x) for x in ece_curves]
    eces_med, eces_lower, eces_upper = np.percentile(ece_curves, 50, axis=0), np.percentile(ece_curves, 2.5, axis=0), np.percentile(ece_curves, 97.5, axis=0)
    return ece_curves, auece_scores, thresholds, eces_med, eces_lower, eces_upper

from sklearn.preprocessing import label_binarize 

def calibration_curve(y_true, y_prob, *, normalize=False, n_bins=5,
                      strategy='uniform'):
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)

    if normalize:  # Normalize predicted values into interval [0, 1]
        y_prob = (y_prob - y_prob.min()) / (y_prob.max() - y_prob.min())
    elif y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError("y_prob has values outside [0, 1] and normalize is "
                         "set to False.")

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError("Only binary classification is supported. "
                         "Provided labels %s." % labels)
    y_true = label_binarize(y_true, classes=labels)[:, 0]

    if strategy == 'quantile':  # Determine bin edges by distribution of data
        quantiles = np.linspace(0, 1, n_bins + 1)
        bins = np.percentile(y_prob, quantiles * 100)
        bins[-1] = bins[-1] + 1e-8
    elif strategy == 'uniform':
        bins = np.linspace(0., 1. + 1e-8, n_bins + 1)
    else:
        raise ValueError("Invalid entry to 'strategy' input. Strategy "
                         "must be either 'quantile' or 'uniform'.")

    binids = np.digitize(y_prob, bins) - 1

    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    nonzero = bin_total != 0
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]

    return prob_true, prob_pred

In [ ]:
# Figure 1 Information

y_true, y_score = master['y'], master['y_score_mcures']
combine_all_scores = [x for x in master["y_scores_mcures_lst_eval1"]]

In [ ]:
# ROC
fpr, tpr, roc_thresholds = metrics.roc_curve(y_true, y_score)
roc_curves, auroc_scores, mean_fpr, tprs_lower, tprs_upper = get_roc_CI(y_true, y_score)

# PR
prec, rec, pr_thresholds = metrics.precision_recall_curve(y_true, y_score)
pr_curves, aupr_scores, mean_rec, prec_lower, prec_upper = get_pr_CI(y_true, y_score)

# Calibration
# combine_all_scores = df_plot['all_predictions'].iloc[i]
all_ys = [[y_true[i]] * len(combine_all_scores[i]) for i in range(len(y_true))]
flat_ys = [item for sublist in all_ys for item in sublist]
flat_scores = [item for sublist in combine_all_scores for item in sublist]
P_true, P_pred = calibration_curve(flat_ys, flat_scores, strategy = 'quantile')
ece_score = np.mean(np.square(np.array(P_true) - np.array(P_pred)))
P_trues, P_preds = get_calibs(combine_all_scores, all_ys)
ece_scores = np.mean(np.square(np.array(P_trues) - np.array(P_preds)), axis=1)

In [ ]:
# Figure 2

months1 = ['3/20', '4/20', '5/20']
months2 = ['6/20', '7/20', '8/20']
months3 = ['9/20', '10/20', '11/20']
months4 = ['12/20', '1/21', '2/21']

Ns, percs, aurocs, auprs = [], [], [], []
for chunks in [months1, months2, months3, months4]: 
    df = master[master['admission_date'].isin(chunks)]
    N, perc = len(df['y']), np.mean(df['y'])
    Ns.append(N)
    percs.append(perc)
    
    if N > 25 and perc > 0 and perc < 1: 
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        aurocs.append((all_rocs, roc))
        auprs.append((all_prs, pr))
    else: 
        aurocs.append([[0], 0])
        auprs.append([[0], 0])

all_months = months1 + months2 + months3 + months4

Ns_months, percs_months, aurocs_months, auprs_months = [], [], [], []
for chunks in all_months: 
    df = master[master['admission_date'].isin([chunks])]
    N, perc = len(df['y']), np.mean(df['y'])
    Ns_months.append(N)
    percs_months.append(perc)
    
    if N > 25 and perc > 0 and perc < 1:
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        aurocs_months.append((all_rocs, roc))
        auprs_months.append((all_prs, pr))
    else:
        aurocs_months.append([[0], 0])
        auprs_months.append([[0], 0])
    
figure2_results = {
    # Across months
    'by_months-Ns': Ns,
    'by_months-percs': percs,
    'by_months-aurocs': aurocs,
    'by_months-auprs': auprs,
    'granular-Ns': Ns_months, 
    'granular-percs': percs_months, 
    'granular-aurocs': aurocs_months, 
    'granular-auprs': auprs_months
}

In [ ]:
by_quarter_names = []

for chunks in [months1, months2, months3, months4]:
    by_quarter_names.append('Quarter:'+str(chunks))
    
by_month_names = []
for month in all_months:
    by_month_names.append('Month:'+month)


In [ ]:
from tqdm import tqdm 

# Across Demographic Subgroups
by_demog_names = []
by_demog_Ns = []
by_demog_percs = []
by_demog_aurocs = []
by_demog_auprs = []

# Sex
for sex in tqdm(np.unique(master['sex']), desc='Sex'):
    df = master[master['sex'] == sex]
    N, perc = len(df['y']), np.mean(df['y'])
    by_demog_names.append('Sex:'+sex)
    by_demog_Ns.append(N)
    by_demog_percs.append(perc)

    if N > 25 and perc > 0 and perc < 1:
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        by_demog_aurocs.append((all_rocs, roc))
        by_demog_auprs.append((all_prs, pr))
    else:
        by_demog_aurocs.append([[0], 0])
        by_demog_auprs.append([[0], 0])


# Age
age_groups = [[17, 25], [25, 45], [45, 65], [65, 85], [85, 1000]]

for age_lower, age_upper in tqdm(age_groups, desc='Age'):
    df = master[(age_lower < master['age']) & (master['age'] <= age_upper)]
    N, perc = len(df['y']), np.mean(df['y'])
    by_demog_names.append('Age:{}-{}'.format(age_lower, age_upper))
    by_demog_Ns.append(N)
    by_demog_percs.append(perc)

    if N > 25 and perc > 0 and perc < 1:
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        by_demog_aurocs.append((all_rocs, roc))
        by_demog_auprs.append((all_prs, pr))
    else:
        by_demog_aurocs.append([[0], 0])
        by_demog_auprs.append([[0], 0])

# Race
master['race_category'] = master['race'].replace(
    ['African American', 'Caucasian', np.nan, 'Patient Refused', 'Unknown', 'American Indian or Alaska Native', 'Native Hawaiian and Other Pacific Islander', 'More than 1'],
    ['Black', 'White', 'Other', 'Other', 'Other', 'Other', 'Other', 'More than 1']).fillna('Other')

for race in tqdm(np.unique(master['race_category']), desc='Race'): 
    df = master[master['race_category'] == race]
    N, perc = len(df['y']), np.mean(df['y'])
    by_demog_names.append('Race:'+race)
    by_demog_Ns.append(N)
    by_demog_percs.append(perc)

    if N > 25 and perc > 0 and perc < 1:
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        by_demog_aurocs.append((all_rocs, roc))
        by_demog_auprs.append((all_prs, pr))
    else:
        by_demog_aurocs.append([[0], 0])
        by_demog_auprs.append([[0], 0])

# Ethnicity
ethnicities = [
    'Ethnicity:Hispanic',
    'Ethnicity:Non-Hispanic',
    'Ethnicity:Unknown',
]

master['ethnicity_category'] = master['ethnicity'].replace(
    ['Hispanic or Latino', 'Non-Hispanic or Latino', 'Unknown', 'Patient Refused'],
    ['Ethnicity:Hispanic','Ethnicity:Non-Hispanic','Ethnicity:Unknown', 'Ethnicity:Unknown']).fillna('Ethnicity:Unknown')


for ethnicity in tqdm(np.unique(master['ethnicity_category']), desc='Ethnicity'): 
    df = master[master['ethnicity_category'] == ethnicity]
    N, perc = len(df['y']), np.mean(df['y'])
    by_demog_names.append(ethnicity)
    by_demog_Ns.append(N)
    by_demog_percs.append(perc)

    if N > 25 and perc > 0 and perc < 1:
        (_, _), (all_rocs, roc), (all_prs, pr) = get_nums(df)
        by_demog_aurocs.append((all_rocs, roc))
        by_demog_auprs.append((all_prs, pr))
    else:
        by_demog_aurocs.append([[0], 0])
        by_demog_auprs.append([[0], 0])


In [ ]:
by_demog_names, by_demog_Ns

In [ ]:
#Figure 4 -- Secondary Use-Case

from tqdm import tqdm 

def bootstrap_fn(i, df, replace = True, val = 48):
    df_Yte_agg = df.sample(frac = 1, replace=replace, random_state=i)
    
    scores = np.sort(df_Yte_agg['secondary_mcures'])
    
    #Find highest threshold with NPV >= 0.95
    for s in scores: 
        curr = df_Yte_agg[df_Yte_agg['secondary_mcures'] <= s]
        if 1 - curr['y_secondary'].mean() >= 0.95: 
            latest = curr
    try: 
        #Percentage of people discharged
        discharged = latest.shape[0] / len(scores)
        num = latest.shape[0]
        #Total number of days spent in the hospital - 2 days -- amount of potential time saved on these examples 
        total_days = np.sum((latest['final_time_min'] / (60 * 24)) - (val / 24))

    except: 
        return 0, 0
    
    return discharged, total_days


def get_roc_CI(df, val):
    discharged, days = zip(*Parallel(n_jobs=10)(delayed(bootstrap_fn)(i, df, val) for i in range(1000)))
    return discharged, days

    
secondary = master[master['in_second_use_case'] == True]

import time 

now = time.time() 

discharged, days = get_roc_CI(secondary, 48)

print(time.time() - now)

In [ ]:
D_results = []
D_results.append({
    # Name of the institution
    'Institution': institution,
    
    
    ### Figure 1: ROC, PR, Calibration
    # Saves the performance metrics and all the points on each curve, including 95% confidence intervals
    
    # ROC
    'roc_curve': [list(fpr), list(tpr), list(roc_thresholds)],
    'roc_curve_CI': [list(mean_fpr), list(tprs_lower), list(tprs_upper)],
    'auroc_score': metrics.roc_auc_score(y_true, y_score),
    'auroc_score_CI': [np.percentile(auroc_scores, 2.5), np.percentile(auroc_scores, 97.5)],
    'auroc_score_bootstrapped': list(auroc_scores),

    # PR
    'pr_curve': [list(prec), list(rec), list(pr_thresholds)],
    'pr_curve_CI': (list(mean_rec), list(prec_lower), list(prec_upper)),
    'aupr_score': metrics.auc(*metrics.precision_recall_curve(y_true, y_score)[1::-1]),
    'aupr_score_CI': [np.percentile(aupr_scores, 2.5), np.percentile(aupr_scores, 97.5)], 
    'aupr_score_bootstrapped': list(aupr_scores),

    # Calibration
    'calibration_curve': [list(P_true), list(P_pred)],
    'calibration_curve_bootstrapped': [np.array(P_trues).tolist(), np.array(P_preds).tolist()],
    'ece_score': ece_score,
    'ece_score_CI': [np.percentile(ece_scores, 2.5), np.percentile(ece_scores, 97.5)], 
    'ece_score_bootstrapped': list(ece_scores),
    
    
    ### Figures 2-3: Subgroup analyses
    # only summary statistics and performance metrics are saved
    # only generate results if N>25 for a subgroup
    
    # Figure 2A: Across months
    'by_month_names': by_month_names,          # descriptive name for each group
    'by_month_Ns': Ns_months,                  # sample size for each group
    'by_month_percs': percs_months,            # outcome rates for each group
    'by_month_aurocs': aurocs_months,          # (list of bootstrap scores, non-bootstrapped score)
    'by_month_auprs': auprs_months,
    
    # Figure 2B: Across quarters
    'by_quarter_names': by_quarter_names,      # ["Quarter:['3/20', '4/20', '5/20']", "Quarter:['6/20', '7/20', '8/20']", "Quarter:['9/20', '10/20', '11/20']", "Quarter:['12/20', '1/21', '2/21']"]
    'by_quarter_Ns': Ns,
    'by_quarter_percs': percs,
    'by_quarter_aurocs': aurocs,
    'by_quarter_auprs': auprs,
    
    # Figure 3: Across demographics            
    'by_demog_names': by_demog_names,          # ['Sex:F', 'Sex:M', 'Age:17-25', 'Age:25-45', 'Age:45-65', 'Age:65-85', 'Age:85-1000', 'Race:Asian', 'Race:Black', 'Race:Other', 'Race:White', 'Ethnicity:Hispanic', 'Ethnicity:Non-Hispanic', 'Ethnicity:Unknown']
    'by_demog_Ns': by_demog_Ns,
    'by_demog_percs': by_demog_percs,
    'by_demog_aurocs': by_demog_aurocs,
    'by_demog_auprs': by_demog_auprs,
    
    
    ### Figure 4: Secondary use case, 1000 bootstrapped values
    # saves two lists of 1000 numbers
    'days_saved_boostraps': list(days),        # Potential bed days saved (length of stay - 2 days)
    'discharged_boostrap': list(discharged),   # Percentage discharged (num people less than decision threshold at which NPV >= 95%)
})

In [ ]:
# Helper function for rounding numbers in nested list
def round_nested_list(lst, num=3):
    if isinstance(lst, str) or isinstance(lst, int):
        return lst
    elif isinstance(lst, float):
        return round(lst, num) # '%.{}f'.format(num) %lst
    else:
        return [round_nested_list(i) for i in lst]

In [ ]:
df_results = pd.DataFrame(D_results)

# Round all numbers to 3 decimal places
for col in df_results.columns:
    df_results[col] = df_results[col].apply(round_nested_list)

df_results.to_csv('{}-RESULTS.csv'.format(institution), index=False, float_format="%.3f")